# ML Parsing #
Initial parsing of the grade submissions. It is meant only for research purposes and not for production environments.

_What it does?_
- Parses the intial csv files (form csv + grades uploaded in csv format)
- Shows basic info of the data
- Connects form submissions with their associated grades and cleans entry with no grades
- Parses the submission for the CSD Department
- Drops invalid values and fixes any other issues with the dataset
- Produces a final dataset with the form submissions and the grades. Each course found in the dataset is represented as a column. The initial dataset produced 111 columns/unique courses*. 

*Some columns refer to the same course but they are with different codes (old/new study program)

**ATTENTION!!: Some outputs have been deleted for privacy reasons**

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## General Parsing ##

In [3]:
submissions_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 16 columns):
Timestamp                                                                   53 non-null object
Ηλικία                                                                      53 non-null int64
Φύλο                                                                        53 non-null object
Σχολή Φοίτησης                                                              53 non-null object
Επέλεξα τη σχολή μου διότι:                                                 53 non-null object
Κατά μέσο όρο την εβδομάδα, διαβάζω:                                        53 non-null object
Μέσα στο εξάμηνο, παρακαλουθώ:                                              53 non-null object
Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;         53 non-null object
Μετά το πτυχίο, θα ήθελα να ακολουθήσω:                                     53 non-null object
Ποιο από τα παρακάτω ισχύει;               

In [4]:
#Create an extract column to store the paths of the csvs with the grades
csv = [None] * (submissions_csv.shape[0])
submissions_csv["CSV"] = pd.Series(csv).to_numpy()
# display(submissions_csv)
# print(submissions_csv['Filename'])

## Parsing for Departments ##

In [7]:
csd = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "Πληροφορική ΑΠΘ"]
hmmy = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "ΗΜΜΥ ΑΠΘ"]
print("CSD:" + str(csd.shape[0]))
print("HMMY:" + str(hmmy.shape[0]))

CSD:39
HMMY:12


### CSD Department - Grade Statistics ###

In [8]:
# Concat all grades (all the grade csvs together)
csd_csv_files = csd['CSV'].to_numpy()
li = []
# pd.read_csv(csd_csv_files[5],encoding='utf-8')
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
# coltypes = {"C_ID":str,"C_NAME":str,"TYPE":str,"EXAM_YEAR":str,"EXAM_PERIOD":str,"ECTS":np.int32,"DM":np.int32,"FACTOR":np.int32,"GRADE":np.float64}
for csv_file in csd_csv_files:
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    li.append(df)

csd_grades = pd.concat(li, axis=0, ignore_index=True)
display(csd_grades)

,C_ID,C_NAME,TYPE,EXAM_YEAR,EXAM_PERIOD,ECTS,DM,FACTOR,GRADE
0,NCO-01-01,ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ Ι,Υ,2017 - 2018,ΦΕΒΡ,5.0,4.0,5.0,10
1,NCO-01-02,ΕΙΣΑΓΩΓΗ ΣΤΗΝ ΠΛΗΡΟΦΟΡΙΚΗ,Υ,2017 - 2018,ΦΕΒΡ,4.0,3.0,4.0,10
2,NCO-01-03,ΓΡΑΜΜΙΚΗ ΑΛΓΕΒΡΑ,Υ,2017 - 2018,ΦΕΒΡ,5.0,4.0,5.0,10
3,NCO-01-04,ΔΙΑΚΡΙΤΑ ΜΑΘΗΜΑΤΙΚΑ,Υ,2017 - 2018,ΦΕΒΡ,7.0,5.0,7.0,9
4,NCO-01-05,ΒΑΣΙΚΕΣ ΑΡΧΕΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ,Υ,2017 - 2018,ΦΕΒΡ,9.0,7.0,9.0,10
...,...,...,...,...,...,...,...,...,...
942,NDM-08-0 3,ΜΟΝΤΕΛΟΠΟΙΗΣΗ - ΨΗΦΙΑΚΗ ΣΥΝΘΕΣΗ ΕΙΚΟΝΩΝ,ΥΕΚ,2018 - 2019,ΦΕΒΡ,5.0,3.0,5.0,7
943,NET-08-02,ΜΟΥΣΙΚΗ ΠΛΗΡΟΦΟΡΙΚΗ,Ε,2018 - 2019,ΙΟΥΝ,5.0,4.0,5.0,9
944,NGE-06-03,ΘΕΜΕΛΙΩΣΕΙΣ ΚΡΥΠΤΟΓΡΑΦΙΑΣ,ΓΕ,2018 - 2019,ΙΟΥΝ,5.0,4.0,5.0,6
945,NIS-08-04,ΜΗΧΑΝΙΚΗ ΜΑΘΗΣΗ,Ε,2018 - 2019,ΙΟΥΝ,5.0,0.0,5.0,7


In [9]:
# Find more information on the courses
unique_courses = csd_grades["C_ID"].unique()
print("Total unique courses:{}".format(len(unique_courses)))
csd_grades['C_ID'] = csd_grades['C_ID'].str.replace(' ','',regex=True)
csd_grades["C_ID"].value_counts()

Total unique courses:101


NCO-01-01    37
NCO-02-05    37
NCO-02-03    37
NCO-02-01    37
NCO-01-02    36
             ..
CO-04-02      1
CO-04-07      1
NDM-06-04     1
NET-08-01     1
NDM-08-03     1
Name: C_ID, Length: 101, dtype: int64

In [10]:
# Fixing Issues
def fix_issues(df):
    
    df['C_ID'] = df['C_ID'].str.replace(' ','',regex=True)
    
    df['C_NAME'] = df['C_NAME'].str.replace('*','',regex=True) # Removing stars
    df['C_NAME'] = df['C_NAME'].str.strip() # Removing leading and trailing spaces

    # Fixing issue with parser
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΤΙΚΕΙΜΕΝΟΣΤΡΕΦΗΣ','ΑΝΤΙΚΕΙΜΕΝΟΣΤΡΕΦΗΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΣ',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ','ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ ΑΥΤΟΜΑΤΩΝ',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΤΕΧΝΙΚΕΣ ΔΟΜΗΣΗΣ','ΤΕΧΝΙΚΕΣ ΔΟΜΗΣΗΣ ΔΕΔΟΜΕΝΩΝ',regex=False)
    
#     # Grouping old courses with new ones with the same name
#     unique_courses = df['C_NAME'].unique()
#     for course in unique_courses:
#         course_grades = df[df['C_NAME'] == course]
#         if course_grades['C_ID'].nunique() > 1:
#             course_codes = course_grades['C_ID'].value_counts().index.values
#             for i in range(1,len(course_codes)):
#                 df['C_ID'] = df['C_ID'].replace(course_codes[i],course_codes[0],regex=False)
                
                
    ## Manual Work ## 
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΑΓΝΩΡΙΣΗ ΠΡΟΤΥΠΩΝ','ΑΝΑΓΝΩΡΙΣΗ ΠΡΟΤΥΠΩΝ -ΣΤΑΤΙΣΤΙΚΗ ΜΑΘΗΣΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('NDM-06-03','NDM-06-04',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΩΤΕΡΑ ΜΑΘΗΜΑΤΙΚΑ Ι','ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ Ι',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-01','NCO-01-01',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΩΤΕΡΑ ΜΑΘΗΜΑΤΙΚΑ ΙΙ','ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ ΙΙ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-01','NCO-02-01',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΕΙΣΑΓΩΓΗ ΣΤΟΥΣ ΥΠΟΛΟΓΙΣΤΕΣ','ΕΙΣΑΓΩΓΗ ΣΤΗΝ ΠΛΗΡΟΦΟΡΙΚΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-02','NCO-01-02',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ ΑΥΤΟΜΑΤΩΝ','ΘΕΩΡΙΑ ΥΠΟΛΟΓΙΣΜΟΥ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-04','NCO-02-05',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΝΕΥΡΩΝΙΚΑ ΔΙΚΤΥΑ','ΝΕΥΡΩΝΙΚΑ ΔΙΚΤΥΑ - ΒΑΘΙΑ ΜΑΘΗΣΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('NDM-07-03','NDM-07-05',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΠΙΘΑΝΟΤΗΤΕΣ KAI ΣΤΑΤΙΣΤΙΚΗ','ΠΙΘΑΝΟΤΗΤΕΣ & ΣΤΑΤΙΣΤΙΚΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-02','NCO-02-02',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΣΧΕΔΙΑΣΗ ΑΛΓΟΡΙΘΜΩΝ','ΑΛΓΟΡΙΘΜΟΙ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-05','NCO-04-03',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΣΧΕΔΙΑΣΗ ΓΛΩΣΣΩΝ ΚΑΙ ΜΕΤΑΓΛΩΤΤΙΣΤΕΣ','ΣΧΕΔΙΑΣΗ ΓΛΩΣΣΩΝ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-03','NCO-04-01',regex=False)
    
    # More manual
    df['C_ID'] = df['C_ID'].replace('CO-01-05','NCO-01-05',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-06','NCO-01-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-03','NCO-01-03',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-05','NCO-02-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-02','NCO-04-05',regex=False)
    
    
    return df

In [11]:
def print_full(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

In [12]:
# Checking more info
unique_names = csd_grades.groupby('C_NAME')['C_ID'].value_counts()
print_full(unique_names)
print("Total:" + str(csd_grades['C_NAME'].nunique()))


C_NAME                                                      C_ID     
 COMPUTER GRAPHICS -ΓΡΑΦΙΚΑ                                 ER-01-02      1
 DISTRIBUTED SYSTEMS -ΚΑΤΑΝΕΜΗΜΕΝΑ ΔΙΑΔΙΚΤΥΑΚΑ ΣΥΣΤΗΜΑΤΑ    ER-01-01      1
 ERASMUS 1                                                  ER-01-01      1
 ERASMUS 2                                                  ER-01-02      1
 INTELLIGENT SYSTEMS                                        ER-01-03      1
 INTRODUCTION TO SWEDISH LEVEL 1                            ER-01-02      1
 MATHEMATICAL STATISTICS                                    ER-01-01      1
 SOFTWARE DEVELOPMENT FOR REAL-TIME SYSTEMS                 ER-01-04      1
 ΑΓΓΛΙΚΑ Ι  *                                               CO-01-07      4
 ΑΓΓΛΙΚΑ ΙV                                                 CO-04-07      1
 ΑΓΓΛΙΚΑ ΙΙ                                                 CO-02-07      1
 ΑΓΓΛΙΚΑ ΙΙ  *                                              CO-02-07      3
 ΑΓΓΛΙΚΑ ΙΙΙ  *   

In [13]:
csd_grades = fix_issues(csd_grades)

In [14]:
unique_names = csd_grades.groupby('C_NAME')['C_ID'].value_counts()
print_full(unique_names)
print("Total:" + str(csd_grades['C_NAME'].nunique()))

C_NAME                                                   C_ID     
COMPUTER GRAPHICS -ΓΡΑΦΙΚΑ                               ER-01-02      1
DISTRIBUTED SYSTEMS -ΚΑΤΑΝΕΜΗΜΕΝΑ ΔΙΑΔΙΚΤΥΑΚΑ ΣΥΣΤΗΜΑΤΑ  ER-01-01      1
ERASMUS 1                                                ER-01-01      1
ERASMUS 2                                                ER-01-02      1
INTELLIGENT SYSTEMS                                      ER-01-03      1
INTRODUCTION TO SWEDISH LEVEL 1                          ER-01-02      1
MATHEMATICAL STATISTICS                                  ER-01-01      1
SOFTWARE DEVELOPMENT FOR REAL-TIME SYSTEMS               ER-01-04      1
ΑΓΓΛΙΚΑ Ι                                                CO-01-07      4
ΑΓΓΛΙΚΑ ΙV                                               CO-04-07      1
ΑΓΓΛΙΚΑ ΙΙ                                               CO-02-07      4
ΑΓΓΛΙΚΑ ΙΙΙ                                              CO-03-07      1
ΑΛΓΟΡΙΘΜΟΙ                                               

In [15]:
import numpy as np
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
unique_courses = csd_grades["C_ID"].unique()
print(unique_courses)
course_dictionary = dict()
# Create a dictionary for each course, each row represents a student
for course in unique_courses:
    course_dictionary[course] = [-1] * len(csd_csv_files) 
for i in range(0,len(csd_csv_files)):
    csv_file = csd_csv_files[i]
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    df = fix_issues(df)
    for course_id in df["C_ID"].to_numpy():
        course_dictionary[course_id][i] =  df[df["C_ID"] == course_id]["GRADE"].to_numpy()[0]
csd_courses_df = pd.DataFrame.from_dict(course_dictionary)
display(csd_courses_df)

['NCO-01-01' 'NCO-01-02' 'NCO-01-03' 'NCO-01-04' 'NCO-01-05' 'NCO-02-01'
 'NCO-02-02' 'NCO-02-03' 'NCO-02-04' 'NCO-02-05' 'NCO-03-01' 'NCO-03-02'
 'NCO-03-03' 'NCO-03-04' 'NCO-03-05' 'NCO-04-01' 'NCO-04-02' 'NCO-04-03'
 'NCO-04-04' 'NCO-04-05' 'NCO-05-01' 'NCO-05-02' 'NCO-05-03' 'NCO-05-04'
 'NCO-05-05' 'NIS-06-01' 'NIS-06-03' 'NIS-06-04' 'NGE-06-02' 'NGE-06-03'
 'NIS-06-05' 'NIS-06-06' 'NNA-06-04' 'NIS-06-07' 'NIS-07-07' 'NIS-07-03'
 'NGE-08-01' 'NIS-07-01' 'NIS-07-02' 'NIS-07-04' 'NIS-07-05' 'NIS-08-02'
 'NGE-07-02' 'CO-01-07' 'CO-01-04' 'CO-02-07' 'CO-02-06' 'CO-02-03'
 'NET-06-03' 'NIS-06-02' 'NNA-06-06' 'NDM-07-04' 'NET-07-01' 'NET-08-02'
 'NET-06-01' 'ER-01-01' 'ER-01-02' 'ER-01-03' 'ER-01-04' 'NIS-08-03'
 'CO-03-07' 'CO-03-05' 'CO-03-03' 'CO-04-04' 'CO-04-07' 'CO-04-06'
 'NET-06-02' 'NNA-06-02' 'NGE-06-01' 'NGE-07-01' 'NDM-07-05' 'NNA-07-06'
 'NNA-07-02' 'NET-07-02' 'NET-07-03' 'NET-08-01' 'NDM-06-01' 'NDM-06-02'
 'NDM-06-04' 'NIS-07-08' 'NDM-07-01' 'NDM-07-02' 'NDM-08-02' 'NGE-

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NCO-02-02,NCO-02-03,NCO-02-04,NCO-02-05,...,NDM-06-02,NDM-06-04,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-08-03,NIS-08-04
0,10,10,10,9,10,8,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
1,6,10,7,8,7,8,5,7,9,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
2,6,7,5,10,9,6,5,6,7,5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
3,7,10,8,9,10,6,8,8,9,9,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
4,10,10,10,10,10,10,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
5,7,10,6,7,10,7,7,8,10,6,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
6,6,9,9,7,9,7,8,7,1,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
7,8,10,10,9,10,10,9,9,10,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
8,4,9,4,6,10,7,,7,9,,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
9,7,10,10,8,10,9,9,9,10,9,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1


In [16]:
# Fill nan values with -1
csd_courses_df.fillna(-1,inplace = True)
# Replace blank values with -1
csd_courses_df.replace(r'^\s*$', -1, regex=True,inplace=True)
# Replace ΕΠΙΤ with 10
csd_courses_df.replace(' ΕΠΙΤ',10,inplace=True)
display(csd_courses_df)

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NCO-02-02,NCO-02-03,NCO-02-04,NCO-02-05,...,NDM-06-02,NDM-06-04,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-08-03,NIS-08-04
0,10,10,10,9,10,8,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
1,6,10,7,8,7,8,5,7,9,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
2,6,7,5,10,9,6,5,6,7,5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
3,7,10,8,9,10,6,8,8,9,9,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
4,10,10,10,10,10,10,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
5,7,10,6,7,10,7,7,8,10,6,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
6,6,9,9,7,9,7,8,7,1,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
7,8,10,10,9,10,10,9,9,10,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
8,4,9,4,6,10,7,-1,7,9,-1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
9,7,10,10,8,10,9,9,9,10,9,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1


In [17]:
# Change to type flaot
csd_courses_df = csd_courses_df.astype(np.float64)

In [18]:
csd_courses_df.describe()

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NCO-02-02,NCO-02-03,NCO-02-04,NCO-02-05,...,NDM-06-02,NDM-06-04,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-08-03,NIS-08-04
count,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,...,39.000000,39.000000,39.000000,39.0,39.000000,39.0,39.000000,39.000000,39.000000,39.000000
mean,6.256410,8.282051,6.487179,5.846154,6.846154,6.307692,6.705128,6.102564,6.576923,5.435897,...,-0.615385,-0.743590,-0.871795,-1.0,-0.794872,-1.0,-0.794872,-0.717949,-0.794872,-0.794872
std,2.807115,2.235464,3.227120,3.166755,3.717232,2.903300,2.310496,3.522819,3.396086,3.201931,...,1.710884,1.312249,0.800641,0.0,1.281025,0.0,1.281025,1.761410,1.281025,1.281025
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
25%,5.000000,7.000000,5.000000,5.000000,5.500000,5.500000,6.000000,6.000000,5.000000,4.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
50%,6.000000,9.000000,7.000000,6.000000,8.000000,7.000000,7.000000,7.000000,8.000000,6.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
75%,8.000000,10.000000,8.500000,8.000000,10.000000,8.000000,8.000000,8.000000,9.000000,7.000000,...,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,8.000000,7.000000,4.000000,-1.0,7.000000,-1.0,7.000000,10.000000,7.000000,7.000000


#### Concatenate CSD Student + Grades ####

In [20]:
# Remove unnecessary fields for final dataset
csd_final = csd.drop(['Timestamp', 'Σχολή Φοίτησης',
       'Υποβολή αναλυτικής βαθμολογίας',
       'Συναινώ στη χρήση των στοιχείων μου από την ομάδα του Grade++',
       'Αν θες να λάβεις πρώτος πρόσβαση στην εφαρμογή, συμπλήρωσε το email σου:',
       'Filename', 'CSV'],axis=1)
csd_final.reset_index(drop=True,inplace=True)
display(csd_final)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες"
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός"
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες"
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός"
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός"
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games"
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες"
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές..."
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες"


In [21]:
# Concat with grades
ldf = [csd_final, csd_courses_df]
csd_final = pd.concat(ldf,axis=1)
display(csd_final)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:,...,NDM-06-02,NDM-06-04,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-08-03,NIS-08-04
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [22]:
# Write to file
csd_final.to_csv("csd.csv",index=False)